In [ ]:
import pandas as pd
import numpy as np

df_peliculas = pd.read_csv("data/dataset_netflix_titles.csv")

# Exploración inicial
df_peliculas.head()           # primeras filas
df_peliculas.info()           # info general, tipos de datos y nulos
df_peliculas.describe()       # descripción de columnas numéricas
df_peliculas.columns         # listado de columnas
df_peliculas['type'].value_counts()  # ejemplo de distribución de columna


In [ ]:
print(df_peliculas.duplicated().sum())  # número de filas duplicadas
df_peliculas = df_peliculas.drop_duplicates(keep='last')  # eliminar duplicados
cardinalidad = {col: df_peliculas[col].nunique(dropna=True) / len(df_peliculas) for col in df_peliculas.columns}

# Revisar posibles columnas como índice (cardinalidad = 100%)
for col, val in cardinalidad.items():
    if val == 1:
        print(f"{col} podría ser índice único")


In [ ]:
card_series = pd.Series({col: df_peliculas[col].nunique() / len(df_peliculas) for col in df_peliculas.columns})
for col in df_peliculas.select_dtypes(include='object').columns:
    df_peliculas[col] = df_peliculas[col].str.strip()
low_card = [col for col in df_peliculas.columns if df_peliculas[col].nunique() / len(df_peliculas) < 0.1]
print(low_card)
# Ejemplo simplificación de país
df_peliculas['country'] = df_peliculas['country'].replace(['USA', 'United States'], 'US')

# Ejemplo de limpieza en 'rating'
df_peliculas['rating'] = df_peliculas['rating'].replace(['UR', 'ALL'], 'G')


In [ ]:
df_peliculas['date_added'] = pd.to_datetime(df_peliculas['date_added'], errors='coerce')
df_peliculas['release_year'] = df_peliculas['release_year'].astype(int)


In [ ]:
cols_nulos = df_peliculas.columns[df_peliculas.isnull().any()]
nulos_pct = df_peliculas[cols_nulos].isnull().mean() * 100


In [ ]:
# Columnas con menor % de nulos
df_peliculas['director'] = df_peliculas['director'].fillna('UNK')
df_peliculas['cast'] = df_peliculas['cast'].fillna('UNK')

# Columnas restantes
for col in cols_nulos:
    df_peliculas[col] = df_peliculas[col].fillna('')
df_peliculas = df_peliculas.drop_duplicates(keep='first')
df_movies = df_peliculas[df_peliculas['type'] == 'Movie'].copy()
df_series = df_peliculas[df_peliculas['type'] == 'TV Show'].copy()
# Películas
df_movies['duration'] = df_movies['duration'].str.replace(' min', '').astype(float)

# Series
df_series['duration'] = df_series['duration'].str.replace(' Season', '').str.replace('s', '').astype(int)
df_series.rename(columns={'duration':'num_seasons'}, inplace=True)


In [ ]:
import re

def es_secuel(title):
    # Busca números al final o '2', '3', 'II', 'III'
    if re.search(r'(\b2\b|\b3\b|II|III)', title):
        return True
    return False

df_movies['Posible_Secuela'] = df_movies['title'].apply(es_secuel)
